## Clasificación usando SVM y Bag of Words con descriptores SIFT

Del capítulo 7 de J.Howse et al. "Learning OpenCV 4 Computer Vision with Python 3" (2020)

In [1]:
import cv2
import numpy as np

In [2]:
def get_pos_and_neg_paths(i):
    pos_path = 'images/car/train/pos-%d.pgm' % (i+1) # Imagen de carro (+)
    neg_path = 'images/car/train/neg-%d.pgm' % (i+1) # Imagen de no carro 
    return pos_path, neg_path

def add_sample(path):
    # Lectura de la imagen
    I = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # Extracción de descriptores SIFT
    keypoints, descriptores = sift.detectAndCompute(I, None)
    # Añadir descriptores al entrenador de vocabulario de BoW
    if descriptores is not None:
        bow_kmeans_trainer.add(descriptores)

def extract_bow_descriptors(img):
    # Toma una imagen y devuelve su vector descriptor de BoW
    features = sift.detect(img)
    return bow_extractor.compute(img, features)

In [3]:
# Número de imágenes de muestra para Bag of Words
BOW_NUM_TRAINING_SAMPLES_PER_CLASS = 10
# Número de descriptores BoW para SVM
SVM_NUM_TRAINING_SAMPLES_PER_CLASS = 100

# Instancia del descriptor SIFT
sift = cv2.xfeatures2d.SIFT_create()
# Uso de FLANN para encontrar las correspondencias
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = {}
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [4]:
# Objeto que entrena un vocabulario de Bag of Words (40 clústeres)
bow_kmeans_trainer = cv2.BOWKMeansTrainer(40)
# Objeto para convertir descriptores SIFT en BoW
bow_extractor = cv2.BOWImgDescriptorExtractor(sift, flann)

# Bucle para algunas imágenes de entrenamiento
for i in range(BOW_NUM_TRAINING_SAMPLES_PER_CLASS):
    pos_path, neg_path = get_pos_and_neg_paths(i)
    add_sample(pos_path)
    add_sample(neg_path)

# Clusterización usando K-means. Retorna el vocabulario.
voc = bow_kmeans_trainer.cluster()
bow_extractor.setVocabulary(voc)

# Extracción de descriptores BoW
training_data = []
training_labels = []
for i in range(SVM_NUM_TRAINING_SAMPLES_PER_CLASS):
    pos_path, neg_path = get_pos_and_neg_paths(i)
    pos_img = cv2.imread(pos_path, cv2.IMREAD_GRAYSCALE)
    pos_descriptors = extract_bow_descriptors(pos_img)
    if pos_descriptors is not None:
        training_data.extend(pos_descriptors)
        training_labels.append(1)
    neg_img = cv2.imread(neg_path, cv2.IMREAD_GRAYSCALE)
    neg_descriptors = extract_bow_descriptors(neg_img)
    if neg_descriptors is not None:
        training_data.extend(neg_descriptors)
        training_labels.append(-1)

In [5]:
# dir(bow_extractor)  # Atributos del objeto "bow_extractor"

# Número de elementos de "box_extractor": 40 clústeres (cada uno con un vector SIFT de 128 elementos)
bow_extractor.getVocabulary().shape

(40, 128)

In [6]:
# Números de elementos de entrenamiento
print("Número de datos de entrenamiento:" , len(training_data))
print("Tamaño de cada elemento de entrenamiento:",  training_data[0].shape)

Número de datos de entrenamiento: 200
Tamaño de cada elemento de entrenamiento: (40,)


In [7]:
# Creación de un SVM
svm = cv2.ml.SVM_create()
# Entrenamiento del SVM
svm.train(np.array(training_data), cv2.ml.ROW_SAMPLE, np.array(training_labels))

True

In [8]:
# Lista de imágenes de prueba
lista_imagenes = ['images/car/test/test-0.pgm',
                  'images/car/test/test-1.pgm',
                  'images/car.jpg',
                  'images/campo.jpg',
                  'images/statue.jpg',
                  'images/woodcutters.jpg']

for test_img_path in lista_imagenes:
    img = cv2.imread(test_img_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    descriptors = extract_bow_descriptors(gray_img)
    prediction = svm.predict(descriptors)
    if prediction[1][0][0] == 1.0:
        text = 'Es auto'
        color = (0, 255, 0)
    else:
        text = 'No es auto'
        color = (0, 0, 255)
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                color, 2, cv2.LINE_AA)
    
    # Se abrirá una nueva ventana (presionar cualquier tecla para continuar)
    cv2.imshow(test_img_path, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()